# MSBD 5012 group project

In [1]:
"""     
Student Name:  ZHANG Xinyue                /        Qiao  Shuyu        /       Li Zuoxuan
Student ID:    20750194                    /          20747563         /        20740917
Student Email: xzhangfa@connect.ust.hk     /   sqiaoac@connect.ust.hk  /   zlify@connect.ust.hk
Course Name:   MSBD5012
URL in github: https://github.com/orange-neng/MSBD5012-Forest-type-prediction-exploration
"""

'     \nStudent Name:  ZHANG Xinyue                /        Qiao  Shuyu        /       Li Zuoxuan\nStudent ID:    20750194                    /          20747563         /        20740917\nStudent Email: xzhangfa@connect.ust.hk     /   sqiaoac@connect.ust.hk  /   zlify@connect.ust.hk\nCourse Name:   MSBD5012\nURL in github: \n'

**Description**
In this project, we predict the forest cover type (the predominant kind of tree cover) from strictly cartographic variables (opposed to remotely sensed data). To finish the classification task, we first analyze the dataset to equip the future data pre-processing and application. Then we select multiple machine learning algorithms from various machine learning packages such as Sklearn, Keras and Spark MLlib , to compare their performance. In this report, the process of data analysis, data cleaning, data normalization and hyperparameter tuning will be described to show how they affect the final classification accuracy.
* Here,this file applies three models form sklearn -LR+RF+LightGBM

**Pipeline preocessing：**
  * preprocessing, drop missing values
  * explortary data analysis
  * split, training and testing datasets
  * standardized the training set or not based on models chosen
  * choose models                                         
  * fit the models for training dataset
  * standard scaler in mean std in training dataset
  * predict and evaluate accuracy scores                    

In [3]:
import numpy as np
import pandas as pd
%matplotlib inline     
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV

# model_selection 
# metrics  accuracy_score 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

#StandardScaler distribution with 0 as center, std = 1 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
#import logistic regression for classification 
from sklearn.ensemble import RandomForestClassifier
#import random forest

In [4]:
train = pd.read_csv('train.csv',index_col='Id')
test=0

In [5]:
train.head()  #features of elevation, slope, distance, etc

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [6]:
train['Cover_Type'].value_counts()   #classification, check for each taget and varibale

7    2160
6    2160
5    2160
4    2160
3    2160
2    2160
1    2160
Name: Cover_Type, dtype: int64

In [7]:
train.drop('Cover_Type',axis=1)  #drop this column

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,0
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,0
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,0
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,0
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15116,2607,243,23,258,7,660,170,251,214,1282,...,0,0,0,0,0,0,0,0,0,0
15117,2603,121,19,633,195,618,249,221,91,1325,...,0,0,0,0,0,0,0,0,0,0
15118,2492,134,25,365,117,335,250,220,83,1187,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train,X_test,Y_train,Y_test=train_test_split(train.drop('Cover_Type',axis=1),#feature columns
                                                train.Cover_Type,test_size=0.3,#target columns
                                                random_state=17)
#training and testing datasets   
#shift+tab 


## Split

In [10]:
X_train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
Id,,,,,,,,,,,,,,,,,,,,,
3757,2302,106,16,379,166,1024,246,218,96,361,...,0,0,0,0,0,0,0,0,0,0
12030,2016,47,27,30,10,277,217,169,70,391,...,0,0,0,0,0,0,0,0,0,0
9431,3227,34,17,725,98,2190,215,201,117,2342,...,0,0,0,0,0,0,0,0,0,0
2727,2784,129,20,212,-38,720,249,223,92,390,...,0,0,0,0,0,0,0,0,0,0
13129,2251,359,12,0,0,1622,202,218,153,984,...,0,0,0,0,0,0,0,0,0,0


## Standard Scaler 

In [10]:
Scalar=StandardScaler() #standardized 
scalar=Scalar.fit(X_train)  #only standardized the training dataset, mean and std for all training, store mean and std in scalar
X_train_sc=scalar.transform(X_train)  # mean and std for all training

In [11]:
print(X_train_sc[:,1].var(),X_train_sc[:,1].mean())  #standardized with 0 mean and 1 variance

1.0 6.71336673998583e-17


## Logistic Regression

In [12]:
logit=LogisticRegression(solver='lbfgs', max_iter=500,
                           random_state=17, n_jobs=4,
                          multi_class='multinomial')
#standardized faster 
#lbfgs for solver, liblinear for less data not for classification, newton not applied for L1 regularization 
#maximium iterators as 500
#n_jobs=4

In [13]:
logit.fit(X_train_sc,Y_train)

LogisticRegression(max_iter=500, multi_class='multinomial', n_jobs=4,
                   random_state=17)

In [14]:
X_test_sc=scalar.transform(X_test)  #also standardized the testing dataset, keep mean and std

In [15]:
Y_pred=logit.predict(X_test_sc)

In [16]:
accuracy_score(Y_test,Y_pred) 

0.7067901234567902

## Random Forest

In [17]:
RF1=RandomForestClassifier(n_estimators=100,random_state=17,n_jobs=2)
#100 estimators
#n_jobs=2

In [18]:
RF1.fit(X_train,Y_train)  #not standardized, tree as partial node 

RandomForestClassifier(n_jobs=2, random_state=17)

In [19]:
Y_pred_rf1=RF1.predict(X_test)

In [20]:
accuracy_score(Y_test,Y_pred_rf1)

0.8602292768959435

In [21]:
#check for standardized has effects or not
RF2=RandomForestClassifier(n_estimators=100,random_state=17,n_jobs=2)
RF2.fit(X_train_sc,Y_train)
Y_pred_rf2=RF2.predict(X_test_sc)
accuracy_score(Y_test,Y_pred_rf2)

0.8608906525573192

In [22]:
#significant of features ordering in the form of tree X_train.columns
pd.DataFrame(RF1.feature_importances_,index=X_train.columns,columns=['Importance']).sort_values(by='Importance',ascending=False)[:10]

,Importance
Elevation,0.221297
Horizontal_Distance_To_Roadways,0.093678
Horizontal_Distance_To_Fire_Points,0.073004
Horizontal_Distance_To_Hydrology,0.062592
Hillshade_9am,0.052744
Vertical_Distance_To_Hydrology,0.052035
Aspect,0.050237
Hillshade_3pm,0.047294
Hillshade_Noon,0.045997
Wilderness_Area4,0.038577


In [23]:
!pip3 install lightgbm -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/usr/local/opt/python@3.8/bin/python3.8 -m pip install --upgrade pip' command.


## LightGBM  

In [24]:
from lightgbm import LGBMClassifier
#LightGBM depends on OpenMP，Apple Clang not support

#brew install libomp

#solve

In [25]:
lgb_clf=LGBMClassifier(random_state=17)

In [26]:
lgb_clf.fit(X_train,Y_train)

LGBMClassifier(random_state=17)

In [27]:
accuracy_score(Y_test,lgb_clf.predict(X_test_sc))

0.23148148148148148

### stage1 para tuning

In [28]:
para_grid={'num_leaves':[7,15,31,63],
          'max_depth':[3,4,5,6,-1]}

In [29]:
grid_searcher=GridSearchCV(estimator=lgb_clf,param_grid=para_grid,cv=5,verbose=1,n_jobs=2)

In [30]:
grid_searcher.fit(X_train,Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   12.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   37.0s finished


GridSearchCV(cv=5, estimator=LGBMClassifier(random_state=17), n_jobs=2,
             param_grid={'max_depth': [3, 4, 5, 6, -1],
                         'num_leaves': [7, 15, 31, 63]},
             verbose=1)

In [31]:
grid_searcher.best_params_,grid_searcher.best_score_

({'max_depth': -1, 'num_leaves': 63}, 0.8471278952542788)

### stage2 convergence

In [32]:
num_iter=200
lgb_clf2=LGBMClassifier(random_state=17,max_depth=-1,num_leaves=63,n_estimator=num_iter,n_jobs=1)
para_grid2={'learning_rate':np.logspace(-3,0,10)}
grid_searcher2=GridSearchCV(estimator=lgb_clf2,param_grid=para_grid2,cv=5,verbose=1,n_jobs=2)
grid_searcher2.fit(X_train,Y_train)
print(grid_searcher2.best_params_,grid_searcher2.best_score_)
print(accuracy_score(Y_test,grid_searcher2.predict(X_test)))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   46.3s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:   49.0s finished


{'learning_rate': 0.46415888336127775} 0.8584658087870896
0.8694885361552028


In [33]:
final_lgb=LGBMClassifier(n_estimator=num_iter,random_state=17,
                         max_depth=-1,num_leaves=63,
                         n_jobs=2,learning_rate=0.2)

In [34]:
final_lgb.fit(X_train,Y_train)

LGBMClassifier(learning_rate=0.2, n_estimator=200, n_jobs=2, num_leaves=63,
               random_state=17)

In [35]:
accuracy_score(Y_test,final_lgb.predict(X_test))

0.8679453262786596

In [36]:
pd.DataFrame(final_lgb.feature_importances_,
             index=X_test.columns,columns=['Importance']).sort_values(by='Importance',ascending=False)[:10]

,Importance
Elevation,6392
Horizontal_Distance_To_Fire_Points,6137
Horizontal_Distance_To_Roadways,6058
Vertical_Distance_To_Hydrology,3751
Horizontal_Distance_To_Hydrology,3658
Aspect,2882
Hillshade_Noon,2817
Hillshade_9am,2791
Hillshade_3pm,2700
Slope,2115
